# Imputation fro Lab Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Start Imputation Lab data")

Start Imputation Lab data


In [3]:
df_impute = pd.read_csv(r'../raw_data/data_full_merge.csv')

In [4]:
df_impute[df_impute['Pre_OP_hormone_hyperprolaktin'] == 1]['PROL'].isna().sum()

11

In [5]:
# TODO: remove gender placeholders
df_impute["Patient_gender"] = 1 # male =1
df_impute.loc[0:300, 'Patient_gender'] = 0 # female =0

In [6]:
def impute_based_on_age(row,column,age_ranges):
    age = row['Patient_age']
    col_value = row[column]
    if pd.isna(col_value):
        for start, end, imputation_value in age_ranges:
            if start <= age <= end:
                return imputation_value
        return np.nan  
    else:
        return col_value
    
def impute_based_on_age_gender(row,column, age_gender_ranges):
    age = row['Patient_age']
    gender = row['Patient_gender']
    col_value = row[column]
    if pd.isna(col_value):
        for age_range in age_gender_ranges:
            if (age_range['age_range'][0] <= age <= age_range['age_range'][1]) and age_range['gender'] == gender:
                return age_range['imputation_value']
    
    return col_value 

## Imputation Hormones

In [7]:
#TODO: impute hyperprolaktin patients, impute ausfälle von anderen hormonen

In [8]:
# TODO: impute geschlechtshormone (Testo und andere wahrscheinlich )

In [9]:
print(df_impute['TEST'].isna().sum())
testo_mean = df_impute['TEST'].mean()
# TODO: change
age_ranges_test = [(df_impute['Patient_age'].min(), 19, testo_mean), (20, 29, testo_mean), (30, 39, testo_mean), (40, 49, testo_mean), (50, 59, testo_mean), (60, 69, testo_mean), (70, df_impute['Patient_age'].max(), testo_mean)]
df_impute['TEST'] = df_impute.apply(impute_based_on_age,args=(['TEST',age_ranges_test]), axis=1)


249


In [10]:
lh_mean = df_impute['LH'].mean()

In [11]:
print(df_impute['LH'].isna().sum())
lh_mean = df_impute['LH'].mean()
# TODO: change
age_ranges_lh = [(df_impute['Patient_age'].min(), 19, lh_mean), (20, 29, lh_mean), (30, 39, lh_mean), (40, 49, lh_mean), (50, 59, lh_mean), (60, 69, lh_mean), (70, df_impute['Patient_age'].max(), lh_mean)]
df_impute['LH'] = df_impute.apply(impute_based_on_age,args=(['LH',age_ranges_lh]), axis=1)


134


In [12]:
print(df_impute['FSH'].isna().sum())
fsh_mean = df_impute['FSH'].mean()
# TODO: change
age_ranges_fsh = [(df_impute['Patient_age'].min(), 19, fsh_mean), (20, 29, fsh_mean), (30, 39, fsh_mean), (40, 49, fsh_mean), (50, 59, fsh_mean), (60, 69, fsh_mean), (70, df_impute['Patient_age'].max(), fsh_mean)]
df_impute['FSH'] = df_impute.apply(impute_based_on_age,args=(['FSH',age_ranges_fsh]), axis=1)


132


In [13]:
print(df_impute['FT4'].isna().sum())
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5794073/ - Table 3 - Mean
age_ranges_ft4 = [(df_impute['Patient_age'].min(), 19, 1.30*12.871), (20, 29, 1.31*12.871), (30, 39, 1.26*12.871), (40, 49, 1.22*12.871), (50, 59, 1.20*12.871), (60, 69, 1.2*12.871), (70, df_impute['Patient_age'].max(), 1.2*12.871)]
df_impute['FT4'] = df_impute.apply(impute_based_on_age,args=(['FT4',age_ranges_ft4]), axis=1)


38


In [14]:
print(df_impute['IGF1'].isna().sum())
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9206165/ - Table 1 - Median
age_ranges_igf1 = [(df_impute['Patient_age'].min(), 25, 265.00*0.131), (26, 30, 222*0.131), (31, 35, 183*0.131  ),
                   (36, 40, 171*0.131), (41, 45, 148*0.131), (46, 50, 129*0.131),
                   (51, 55, 129.5*0.131),(56, 60, 130*0.131),(61, 65, 129.5*0.131),(66, 70, 128*0.131)
                   ,(71, 75, 123*0.131),(76, df_impute['Patient_age'].max(), 109*0.131)]
df_impute['IGF1'] = df_impute.apply(impute_based_on_age,args=(['IGF1',age_ranges_igf1]), axis=1)


59


In [15]:
print(df_impute['COR60'].isna().sum())
# regular value for cortisol is at least 500 nmol/l
df_impute['COR60']= df_impute['COR60'].fillna(500)

466


In [16]:
print(df_impute['PROL'].isna().sum())
# https://labosud.fr/wp-content/uploads/sites/7/2023/08/Verification_of_Roche_reference_ranges_for_serum_prolactin_in_children_adolescents_adults_and_the_elderly-2023-07-11-05-21.pdf - Table 2 - Median by age and gender
age_gender_ranges_prol = [
    {'age_range': (df_impute['Patient_age'].min(), 9), 'gender': 0, 'imputation_value': 194.8},
    {'age_range': (10, 12), 'gender': 0, 'imputation_value': 273.9},
    {'age_range': (13, 16), 'gender': 0, 'imputation_value': 352.6},
    {'age_range': (17, 19), 'gender': 0, 'imputation_value': 397.},
    {'age_range': (20, 30), 'gender': 0, 'imputation_value': 356.5},
    {'age_range': (31, 40), 'gender': 0, 'imputation_value': 325.4},
    {'age_range': (41, 50), 'gender': 0, 'imputation_value': 292.9},
    {'age_range': (51, 60), 'gender': 0, 'imputation_value': 272.4},
    {'age_range': (61, 70), 'gender': 0, 'imputation_value': 227.9},
    {'age_range': (71, df_impute['Patient_age'].max()), 'gender': 0, 'imputation_value': 250.3},
    
    {'age_range': (df_impute['Patient_age'].min(), 10), 'gender': 1, 'imputation_value': 211.2},
    {'age_range': (11, 13), 'gender': 1, 'imputation_value': 211.2},
    {'age_range': (14, 16), 'gender': 1, 'imputation_value': 262.0},
    {'age_range': (17, 19), 'gender': 1, 'imputation_value': 283.8},
    {'age_range': (20, 30), 'gender': 1, 'imputation_value': 291.7},
    {'age_range': (31, 40), 'gender': 1, 'imputation_value': 260.5},
    {'age_range': (41, 50), 'gender': 1, 'imputation_value': 252.5},
    {'age_range': (51, 60), 'gender': 1, 'imputation_value': 241.},
    {'age_range': (61, 70), 'gender': 1, 'imputation_value': 232.2},
    {'age_range': (71, df_impute['Patient_age'].max()), 'gender': 1, 'imputation_value': 252.4},
    ]

df_impute['PROL'] = df_impute.apply(impute_based_on_age_gender,args=(['PROL',age_gender_ranges_prol]), axis=1)


33


In [17]:
print(f'Sparsity of lab values in percent is:\n{df_impute.iloc[:,2:9].isna().mean()*100}')

Sparsity of lab values in percent is:
Date_Case         0.0
ID_MRI_Machine    0.0
COR60             0.0
FSH               0.0
FT4               0.0
IGF1              0.0
LH                0.0
dtype: float64


In [18]:
df_impute.to_csv(r'../raw_data/data_imputed.csv',index=False)

In [19]:
print("End Imputation Lab data")

End Imputation Lab data
